In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics
#load the training data
a = pd.read_csv('/Users/akhileshsk/Dropbox/Kaggle_competitions/Quora competition/train.csv',delimiter = ',')
a = a.dropna()
q1 = a['question1'].values
q2 = a['question2'].values
labels = a['is_duplicate'].values
#load embeddings and dictionary
final_embeddings = np.load("final_embeddings.npy")
dictionary = np.load('dictionary.npy').item()

def get_word(word):
    if word not in dictionary:
        return 'UNK'
    else:
        return word
specialstr = '?:!/;\|~`%1234567890.,%&$()_{}[]^"'   
#checki if words are in dictionary first
question1_data = []
question2_data = []
count = 0
for j in range(len(q1)):
    if j%50000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q1[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    #print(count)
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    auto_data = np.array(auto_data)
    question1_data.append(auto_data)
    count+=1
    
question1_data =  np.array(question1_data)

for j in range(len(q2)):
    if j%50000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q2[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    #parent_q2.append([auto_data,j])
    auto_data = np.array(auto_data)
    question2_data.append(auto_data)
    
question2_data =  np.array(question2_data)

In [ ]:
pos = []
for i in range(len(question1_data)):
    if question1_data[i].shape[0]==0 or question2_data[i].shape[0]==0 or question1_data[i].shape[0]==1 or question2_data[i].shape[0]==1 or question1_data[i].shape[0]==2 or question2_data[i].shape[0]==2:
        pos.append(i)
    question1_data[i] = np.round(question1_data[i],4)
    question2_data[i] = np.round(question2_data[i],4)
labels = np.delete(labels,pos)
question1_data = np.delete(question1_data,pos)
question2_data = np.delete(question2_data,pos)

lengths1 = [i.shape[0] for i in question1_data]
lengths12 = [i.shape[0] for i in question2_data]

q1_f = np.ones((len(question1_data),3,128),np.float32)
q1_f_cos =np.ones((len(question1_data),3,128),np.float32) 
q1_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    f1 = np.sum(question1_data[i],0)
    f2 = np.sum(question1_data[i][:-1],0)
    f3 = question1_data[i][-1]
    f4 = np.sum(question1_data[i][1:],0)
    f5 = question1_data[i][0]
    q1_f[i] = [f1,f2,f3]
    q1_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q1_f_plane[i] = [f1,f2,f3,f4,f5]

q2_f = np.ones((len(question2_data),3,128),np.float32)
q2_f_cos =np.ones((len(question1_data),3,128),np.float32)
q2_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    f1 = np.sum(question2_data[i],0)
    f2 = np.sum(question2_data[i][:-1],0)
    f3 = question2_data[i][-1]
    f4 = np.sum(question2_data[i][1:],0)
    f5 = question2_data[i][0]
    q2_f[i] = [f1,f2,f3]
    q2_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q2_f_plane[i] = [f1,f2,f3,f4,f5]
    
dist = np.ones((len(q1_f),25),np.float32)
dist_cos = np.ones((len(q1_f),9),np.float32)
for i in range(len(q1_f_plane)):
    dist[i] = sklearn.metrics.pairwise.pairwise_distances(q1_f_plane[i],q2_f_plane[i]).reshape(25)
    dist_cos[i] = sklearn.metrics.pairwise.cosine_distances(q1_f_cos[i],q2_f_cos[i]).reshape(9)
feature_mat = np.hstack((dist,dist_cos))

In [ ]:
#training
import xgboost as xgb
from sklearn.model_selection import train_test_split
X = feature_mat
y = labels
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.33, random_state=53)
dtrain = xgb.DMatrix(train_x,label = train_y)
dvalid = xgb.DMatrix(valid_x,label = valid_y)

#logloss
from sklearn.metrics import log_loss
dtrain = xgb.DMatrix(train_x,label = train_y)
dvalid = xgb.DMatrix(valid_x,label = valid_y)
param = {'max_depth':25, 'eta':0.3, 'silent':0, 'objective':'binary:logistic','subsample':0.5,'gamma':0 }
param['nthread'] = 4
param['eval_metric'] = 'auc'
#param['scale_pos_weight'] = ratio
num_round = 50
evallist  = [(dvalid,'eval'),(dtrain,'train')]
bst = xgb.train( param, dtrain, num_round, evallist)
loss = log_loss(valid_y,bst.predict(dvalid))
print('log loss '+str(loss))
#save model
#bst.save_model('05_07_1AM.model')


In [ ]:
#test cases
#putting together the test matrix
b = pd.read_csv('/Users/akhileshsk/Dropbox/Kaggle_competitions/Quora competition/test.csv',delimiter = ',')
b.ix[b['question1'].isnull(),['question1','question2']] = 'random empty question'
b.ix[b['question2'].isnull(),['question1','question2']] = 'random empty question'
q1 = b['question1'].values
q2 = b['question2'].values

In [ ]:
del question1_data
del question2_data
#check if words are in dictionary first
question1_data = []
question2_data = []
count = 0
for j in range(len(q1)):
    if j%100000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q1[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    #print(count)
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    auto_data = np.array(auto_data)
    question1_data.append(auto_data)
    count+=1
    
question1_data =  np.array(question1_data)

for j in range(len(q2)):
    if j%100000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q2[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    #parent_q2.append([auto_data,j])
    auto_data = np.array(auto_data)
    question2_data.append(auto_data)
    
question2_data =  np.array(question2_data)

pos = []
for i in range(len(question1_data)):
    if question1_data[i].shape[0]==0 or question2_data[i].shape[0]==0 or question1_data[i].shape[0]==1 or question2_data[i].shape[0]==1 or question1_data[i].shape[0]==2 or question2_data[i].shape[0]==2:
        pos.append(i)
question1_data = np.delete(question1_data,pos)
question2_data = np.delete(question2_data,pos)

del valid_y

#q1_f = np.ones((len(question1_data),3,128),np.float32)
q1_f_cos =np.ones((len(question1_data),3,128),np.float32) 
q1_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    if i%100000==0:
        print(i)
    f1 = np.sum(question1_data[i],0)
    f2 = np.sum(question1_data[i][:-1],0)
    f3 = question1_data[i][-1]
    f4 = np.sum(question1_data[i][1:],0)
    f5 = question1_data[i][0]
    #q1_f[i] = [f1,f2,f3]
    q1_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q1_f_plane[i] = [f1,f2,f3,f4,f5]

#q2_f = np.ones((len(question2_data),3,128),np.float32)
q2_f_cos =np.ones((len(question1_data),3,128),np.float32)
q2_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    if i%100000==0:
        print(i)
    f1 = np.sum(question2_data[i],0)
    f2 = np.sum(question2_data[i][:-1],0)
    f3 = question2_data[i][-1]
    f4 = np.sum(question2_data[i][1:],0)
    f5 = question2_data[i][0]
    #q2_f[i] = [f1,f2,f3]
    q2_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q2_f_plane[i] = [f1,f2,f3,f4,f5]
    
import sklearn.metrics
dist = np.ones((len(q1_f_plane),25),np.float32)
dist_cos = np.ones((len(q1_f_plane),9),np.float32)
for i in range(len(q1_f_plane)):
    dist[i] = sklearn.metrics.pairwise.pairwise_distances(q1_f_plane[i],q2_f_plane[i]).reshape(25)
    dist_cos[i] = sklearn.metrics.pairwise.cosine_distances(q1_f_cos[i],q2_f_cos[i]).reshape(9)
feature_mat = np.hstack((dist,dist_cos))

position = pos
dtest = xgb.DMatrix(feature_mat)
results = bst.predict(dtest)
diff_inds = np.setdiff1d(np.arange(b.shape[0]),pos)

result2 = np.zeros(b.shape[0],np.float32)
result2[diff_inds] = results

#%% create a submission

submissionName = 'word_2vec_vectors_xgb'

submission = pd.DataFrame()
submission['test_id'] = b['test_id']
submission['is_duplicate'] = result2
submission.to_csv(submissionName + '.csv', index=False)